In [1]:
import torch
import tensorflow as tf
import numpy as np

from transformers import TFT5EncoderModel, TFT5ForConditionalGeneration, TFT5Model
from transformers import T5Config

In [3]:
from tokenization_enc_dec import EncDecTokenizer
tokenizer = EncDecTokenizer('./vocab.txt')

In [4]:
config = T5Config(
    vocab_size=26240,
#     n_positions=self.n_positions,
    d_model=4096,
    d_ff=10240,
    d_kv=4096 // 64,
    num_layers=24,
    num_heads=64,
    relative_attention_num_buckets=32,
    dropout_rate=0.0,
    initializer_factor=1.0,
    eos_token_id=tokenizer.eod_id,
    bos_token_id=tokenizer.pad_id,
    pad_token_id=tokenizer.pad_id,
    decoder_start_token_id=tokenizer.pad_id,
    feed_forward_proj='gated-gelu',
    tie_word_embeddings=False
)

In [5]:
model = TFT5ForConditionalGeneration(config)

In [6]:
_ = model(input_ids=tf.constant([[1]]), decoder_input_ids=tf.constant([[1]]))

In [7]:
len(model.variables)

558

In [8]:
def get_weight(name):
    return state_dict[name].numpy()


def get_block_weight(n, t='encoder', name=False, dim=4096):
    weights = []
    for k, v in state_dict.items():
        if t in k and f'blocks.{n}.' in k:
            # pytorch和tensorflow版本的weights是矩阵转置的
            w = v.numpy()
            if 'self_attn.project' in k:
                w0, w1, w2 = w[:dim, :], w[dim:dim*2, :], w[dim*2:, :]
                w0 = np.transpose(w0)
                w1 = np.transpose(w1)
                w2 = np.transpose(w2)
                weights.append((k, w0))
                weights.append((k, w1))
                weights.append((k, w2))
            elif 'cross_attn.project_q' in k:
                w = np.transpose(w)
                weights.append((k, w))
            elif 'cross_attn.project_kv' in k:
                w0, w1 = w[:dim, :], w[dim:, :]
                w0 = np.transpose(w0)
                w1 = np.transpose(w1)
                weights.append((k, w0))
                weights.append((k, w1))
            else:
                if 'dense' in k:
                    w = np.transpose(w)
                weights.append((k, w))
    if 'relative_attention_bias' in weights[3][0]:
        weights = weights[3:4] + weights[:3] + weights[4:]
    if not name:
        weights = [x[1] for x in weights]
    return weights

In [9]:
state_dict = torch.load('../converted.zip')

In [10]:
model_new_weights = [get_weight('word_embeds.weight')]
for i in range(24):
    model_new_weights += get_block_weight(i, t='encoder')
model_new_weights += [get_weight('encoder.final_layernorm.weight')]
for i in range(24):
    model_new_weights += get_block_weight(i, t='decoder')

model_new_weights += [get_weight('decoder.final_layernorm.weight')]
model_new_weights += [np.transpose(get_weight('lm_head.weight'))]

In [11]:
len(model_new_weights)

558

In [12]:
len(model.variables)

558

In [13]:
for k, v in state_dict.items():
    print(k, v.shape)

word_embeds.weight torch.Size([26240, 4096])
lm_head.weight torch.Size([26240, 4096])
encoder.word_embeds.weight torch.Size([26240, 4096])
encoder.final_layernorm.weight torch.Size([4096])
encoder.blocks.0.self_attn.self_attn.project.weight torch.Size([12288, 4096])
encoder.blocks.0.self_attn.self_attn.relative_attention_bias.weight torch.Size([32, 64])
encoder.blocks.0.self_attn.self_attn.dense.weight torch.Size([4096, 4096])
encoder.blocks.0.self_attn.layer_norm.weight torch.Size([4096])
encoder.blocks.0.ff.dense_relu_dense.wi_0.weight torch.Size([10240, 4096])
encoder.blocks.0.ff.dense_relu_dense.wi_1.weight torch.Size([10240, 4096])
encoder.blocks.0.ff.dense_relu_dense.wo.weight torch.Size([4096, 10240])
encoder.blocks.0.ff.layer_norm.weight torch.Size([4096])
encoder.blocks.1.self_attn.self_attn.project.weight torch.Size([12288, 4096])
encoder.blocks.1.self_attn.self_attn.dense.weight torch.Size([4096, 4096])
encoder.blocks.1.self_attn.layer_norm.weight torch.Size([4096])
encoder.

In [14]:
for k in model_new_weights:
    print(k.shape)

(26240, 4096)
(32, 64)
(4096, 4096)
(4096, 4096)
(4096, 4096)
(4096, 4096)
(4096,)
(4096, 10240)
(4096, 10240)
(10240, 4096)
(4096,)
(4096, 4096)
(4096, 4096)
(4096, 4096)
(4096, 4096)
(4096,)
(4096, 10240)
(4096, 10240)
(10240, 4096)
(4096,)
(4096, 4096)
(4096, 4096)
(4096, 4096)
(4096, 4096)
(4096,)
(4096, 10240)
(4096, 10240)
(10240, 4096)
(4096,)
(4096, 4096)
(4096, 4096)
(4096, 4096)
(4096, 4096)
(4096,)
(4096, 10240)
(4096, 10240)
(10240, 4096)
(4096,)
(4096, 4096)
(4096, 4096)
(4096, 4096)
(4096, 4096)
(4096,)
(4096, 10240)
(4096, 10240)
(10240, 4096)
(4096,)
(4096, 4096)
(4096, 4096)
(4096, 4096)
(4096, 4096)
(4096,)
(4096, 10240)
(4096, 10240)
(10240, 4096)
(4096,)
(4096, 4096)
(4096, 4096)
(4096, 4096)
(4096, 4096)
(4096,)
(4096, 10240)
(4096, 10240)
(10240, 4096)
(4096,)
(4096, 4096)
(4096, 4096)
(4096, 4096)
(4096, 4096)
(4096,)
(4096, 10240)
(4096, 10240)
(10240, 4096)
(4096,)
(4096, 4096)
(4096, 4096)
(4096, 4096)
(4096, 4096)
(4096,)
(4096, 10240)
(4096, 10240)
(10240, 4

In [15]:
for x in model.variables:
    print(x.name, x.shape)

shared/shared/weight:0 (26240, 4096)
tf_t5for_conditional_generation/encoder/block_._0/layer_._0/SelfAttention/relative_attention_bias/embeddings:0 (32, 64)
tf_t5for_conditional_generation/encoder/block_._0/layer_._0/SelfAttention/q/kernel:0 (4096, 4096)
tf_t5for_conditional_generation/encoder/block_._0/layer_._0/SelfAttention/k/kernel:0 (4096, 4096)
tf_t5for_conditional_generation/encoder/block_._0/layer_._0/SelfAttention/v/kernel:0 (4096, 4096)
tf_t5for_conditional_generation/encoder/block_._0/layer_._0/SelfAttention/o/kernel:0 (4096, 4096)
tf_t5for_conditional_generation/encoder/block_._0/layer_._0/layer_norm/weight:0 (4096,)
tf_t5for_conditional_generation/encoder/block_._0/layer_._1/DenseReluDense/wi_0/kernel:0 (4096, 10240)
tf_t5for_conditional_generation/encoder/block_._0/layer_._1/DenseReluDense/wi_1/kernel:0 (4096, 10240)
tf_t5for_conditional_generation/encoder/block_._0/layer_._1/DenseReluDense/wo/kernel:0 (10240, 4096)
tf_t5for_conditional_generation/encoder/block_._0/layer_

In [16]:
assert len(model_new_weights) == len(model.variables)

In [17]:
model.set_weights(model_new_weights)

In [86]:
input_text = '''当地时间9月6日是美国劳工节，但就在这一天，上千万美国劳动者却陷入新的困境。因为美国政府为疫情期间失业者提供的主要救助同日到期，而且白宫表示没有进一步延长救助的计划。
在德尔塔变异株已把美国推入新一轮疫情的背景下，失业救济的突然“断供”意味着有上千万美国人将全部或部分失去他们的生活来源。'''
input_ids = tf.constant([tokenizer.encode(input_text) + [tokenizer.get_sentinel_id(0)]])

In [91]:
out = model.generate(
    input_ids,
    max_length=50,
    decoder_start_token_id=1,
    do_sample=True,
    top_p=0.95,
    top_k=20,
    bad_words_ids=[[x] for x in range(26050, tokenizer.vocab_size)]
)

In [92]:
out_text = tokenizer.decode(out.numpy()[0].tolist()[1:])

In [93]:
print(out_text)

点点，而且在美国劳动者自己已经为失救而失“困”境的生活中，他们也必须“自力”地工作。
美国劳苦功 夫
节的劳工节当天


In [53]:
input_text = '''当地时间9月6日是美国劳工节，但就在这一天，上千万美国劳动者却陷入新的困境。因为美国政府为疫情期间失业者提供的主要救助同日到期，而且白宫表示没有进一步延长救助的计划。
在德尔塔变异株已把美国推入新一轮疫情的背景下，失业救济的突然“断供”意味着有上千万美国人将全部或部分失去他们的生活来源。'''
output_text = '''美国'''
input_ids = tf.constant([tokenizer.encode(input_text)])

for i in range(20):
    decoder_input_ids = tf.constant([[1, tokenizer.get_sentinel_id(0)] + tokenizer.encode(output_text)])
    out = model(input_ids, decoder_input_ids=decoder_input_ids)
    t = out['logits'][:, -1, :26050].numpy()
    next_token = tokenizer.decode([np.argmax(t, -1)[0]])
    # print(next_token)
    output_text += next_token
    print(output_text)

美国劳动
美国劳动者
美国劳动者的
美国劳动者的新
美国劳动者的新困
美国劳动者的新困境
美国劳动者的新困境。
美国劳动者的新困境。

美国劳动者的新困境。
美国
美国劳动者的新困境。
美国政府
美国劳动者的新困境。
美国政府为
美国劳动者的新困境。
美国政府为疫情
美国劳动者的新困境。
美国政府为疫情期
美国劳动者的新困境。
美国政府为疫情期间
美国劳动者的新困境。
美国政府为疫情期间失业
美国劳动者的新困境。
美国政府为疫情期间失业者
美国劳动者的新困境。
美国政府为疫情期间失业者提供
美国劳动者的新困境。
美国政府为疫情期间失业者提供的
美国劳动者的新困境。
美国政府为疫情期间失业者提供的主要
美国劳动者的新困境。
美国政府为疫情期间失业者提供的主要救助
